# Data Exploration

This notebook is meant to be a template for explorion of the data collected from trials.

### Please duplicate the notebook to start your work and keep the template intact.

## Before running the notebook
The trial datastore process must be started prior to running the notebook. Make sure you open a terminal, activate the .venv environment, run:
```
python -m launch_local_services
```

Once you are done with working in this notebook, close the terminal window.

## Cogment Setup

In [ ]:
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(ROOT_DIR)

from cogment_verse.utils.generate import generate
from cogment_verse.app import SPEC_FILEPATH

WORK_DIR = os.path.join(ROOT_DIR, ".cogment_verse")

generate(WORK_DIR, SPEC_FILEPATH)

In [ ]:
import asyncio
import functools
import re
from dataclasses import dataclass, field
from datetime import datetime
from typing import List

import cogment
import numpy as np
import pandas as pd
from cogment.control import TrialState
from tabulate import tabulate

import gymnasium


from cogment_verse.specs import (
    AgentConfig,
    cog_settings,
    EnvironmentConfig,
    EnvironmentSpecs,
    HUMAN_ACTOR_IMPL,
    PLAYER_ACTOR_CLASS,
    TEACHER_ACTOR_CLASS,
    WEB_ACTOR_NAME,
)
from cogment_verse.specs.ndarray_serialization import deserialize_ndarray, serialize_ndarray
from cogment.session import ActorInfo


TRIAL_DATASTORE_ENDPOINT = "grpc://localhost:9001"

In [ ]:
from pettingzoo.mpe import simple_tag_v3

env = simple_tag_v3.env(
    num_good=2,
    num_adversaries=2,
    num_obstacles=2,
    max_cycles=25,
    continuous_actions=False,
    render_mode="rgb_array",
)

env.reset()

In [ ]:
print(env.agents)
print(env.num_agents)
print(env.observation_space("agent_0"))
print(env.action_space("agent_0"))

In [ ]:
import matplotlib.pyplot as plt

rendered_frame = env.render()

plt.imshow(rendered_frame)

In [ ]:
frames = []
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    action = env.action_space(agent).sample() # this is where you would insert your policy
    print(f"agent: {agent} | observation: {observation.shape} | action: {action}")
    #print(f"\t observation: {observation.shape}")
    #print(f"\t action: {action}")
    frames.append(env.render())
    env.step(action)
    plt.imshow(rendered_frame)

In [ ]:
plt.imshow(frames[100])

In [ ]:
frames[100]

In [ ]:
from enum import Enum


class MpeSpecType(Enum):
    """ Used to associate different environment specs to different actors.
    """
    DEFAULT = "player"
    GOOD = "mpe_agent"
    ADVERSARY = "mpe_adversary"

    @classmethod
    def from_config(cls, spec_type_str: str):
        try:
            return cls(spec_type_str)
        except ValueError:
            raise ValueError(f"Actor specs type ({spec_type_str}) is not a supported type: [{', '.join(MpeSpecType.values)}]")

    @classmethod
    @property
    def values(self) -> List[str]:
        """ Return list of all values available in the enum """
        return list(spec_type.value for spec_type in MpeSpecType)

In [ ]:
l = [1, 2, 3, 4]
a = ["a", "b", "c", "d"]

d = {key: val if val > 2 else 10 for key, val in zip(a, l)}
d

In [ ]:
GOOD_ACTOR_PREFIX = "agent"
ADVERSARY_ACTOR_PREFIX = "adversary"
WEB_ACTOR_NAME = "web_actor"

def get_strings_with_prefix(strings, prefix):
    pattern = r'^' + prefix
    matches = [string for string in strings if re.match(pattern, string)]
    return matches

env_agents = ['adversary_0', 'adversary_1', 'adversary_2', 'adversary_3', 'agent_0', 'agent_1', 'agent_2']

# player_actors = [
#     (0, 'agent_0', 'mpe_agent'), 
#     (1, 'agent_1', 'mpe_agent'), 
#     (2, 'web_actor', 'player'), 
#     (3, 'adversary_0', 'mpe_adversary'), 
#     (4, 'adversary_1', 'mpe_adversary'), 
#     (5, 'adversary_2', 'mpe_adversary'), 
#     (6, 'adversary_3', 'mpe_adversary'),
# ]

player_actors = [
    (0, 'agent_0', 'mpe_agent'), 
    (1, 'agent_1', 'mpe_agent'), 
    (2, 'adversary_0', 'mpe_adversary'), 
    (3, 'adversary_1', 'mpe_adversary'), 
    (4, 'adversary_2', 'mpe_adversary'), 
    (5, 'web_actor', 'player'), 
]

pz_name_to_idx = {agent_name: count for (count, agent_name) in enumerate(env_agents)}
pz_idx_to_names = {v: k for k, v in pz_name_to_idx.items()}

print(f"pz_idx_to_names: {pz_idx_to_names}")
print(f"pz_name_to_idx: {pz_name_to_idx}")

pz_to_actor_idx_mapping = {}
for _actor_idx, _actor_name, _actor_class_name in player_actors:
    if _actor_name == WEB_ACTOR_NAME:
        if _actor_class_name == MpeSpecType.ADVERSARY.value:
            current_names = [pz_idx_to_names[idx] for idx, _ in pz_to_actor_idx_mapping.items()]
            idx_count = len(get_strings_with_prefix(strings=current_names, prefix=ADVERSARY_ACTOR_PREFIX))
            pz_web_actor_name = f"{ADVERSARY_ACTOR_PREFIX}_{idx_count}"
        else:
            current_names = [pz_idx_to_names[idx] for idx, _ in pz_to_actor_idx_mapping.items()]
            idx_count = len(get_strings_with_prefix(strings=current_names, prefix=GOOD_ACTOR_PREFIX))
            pz_web_actor_name = f"{GOOD_ACTOR_PREFIX}_{idx_count}"

        pz_web_actor_idx = pz_name_to_idx[pz_web_actor_name]
        
    else:
        pz_web_actor_idx = pz_name_to_idx[_actor_name]
      
    pz_to_actor_idx_mapping[pz_web_actor_idx] = _actor_idx
    print(f"pz_to_actor_idx_mapping: {pz_to_actor_idx_mapping}")
    current_names = [pz_idx_to_names[idx] for idx, _ in pz_to_actor_idx_mapping.items()]
    print(current_names)

print(f"pz_to_actor_idx_mapping: {pz_to_actor_idx_mapping}")


class MpeActors:
    
    def __init__(self, pz_actors: List[str], cogment_actors: List[ActorInfo]):
        self.pz_actors = pz_actors
        self.player_actors = [
            (actor_idx, actor.actor_name, actor.actor_class_name)
            for (actor_idx, actor) in enumerate(actors)
            if actor.actor_class_name in self.player_classes
        ]